## Setup

In [12]:
import os
from __future__ import annotations
from sparql_prompts import *    
from azure.core.credentials import AzureKeyCredential
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings    
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.base import Chain
from langchain.chains.llm import LLMChain
from langchain_core.callbacks.manager import CallbackManager, CallbackManagerForChainRun
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts.base import BasePromptTemplate
from pydantic import Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

from langchain_community.chains.graph_qa.prompts import (
    GRAPHDB_QA_PROMPT,
    GRAPHDB_SPARQL_FIX_PROMPT,
    GRAPHDB_SPARQL_GENERATION_PROMPT,
)
from langchain_community.graphs import OntotextGraphDBGraph
from IPython.display import Image, display

from langsmith import Client
from langsmith import traceable

from langchain_community.graphs import OntotextGraphDBGraph
from typing import TYPE_CHECKING, Any, Dict, List, Optional
if TYPE_CHECKING:
    import rdflib
    
from rdflib import Graph    
from rdflib.plugins.sparql import prepareQuery
from rdflib import Namespace

import textwrap
import pandas as pd
import requests
from typing import List, OrderedDict
import json

from dotenv import load_dotenv
load_dotenv(os.path.join("..", "Azure OpenAI credentials.env"))

True

In [13]:
azure_endpoint = os.environ['GLOBAL_AZURE_ENDPOINT']
openai_api_key = os.environ['GLOBAL_OPENAI_API_KEY']

openai_deployment_name = os.environ['GLOBAL_GPT_DEPLOYMENT_NAME']
openai_api_version = os.environ['GLOBAL_OPENAI_API_VERSION']
embedding_model = os.environ['GLOBAL_EMBEDDING_MODEL']
embedding_deployment_name = os.environ['GLOBAL_EMBEDDING_DEPLOYMENT_NAME']

search_endpoint = os.environ['SEARCH_ENDPOINT']
search_api_key = os.environ['SEARCH_API_KEY']
search_api_version = os.environ['SEARCH_API_VERSION']
search_service_name = os.environ['SEARCH_SERVICE_NAME']
search_sem_config = os.environ['SEARCH_SEMANTIC_CONFIG_NAME']

# langsmith_api_key = os.environ['LANGSMITH_API_KEY']

search_url = f"https://{search_service_name}.search.windows.net/"
search_credential = AzureKeyCredential(search_api_key)

llm = AzureChatOpenAI(
    deployment_name=openai_deployment_name, 
    openai_api_version=openai_api_version, 
    openai_api_key=openai_api_key, 
    azure_endpoint=azure_endpoint, 
    temperature=0
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=embedding_deployment_name,
    api_version=openai_api_version,
    api_key=openai_api_key,
    azure_endpoint=azure_endpoint,
)

doc_index_name: str = "crd-vector-store"
ontology_index_name: str = "crd-ontologies-desc"


## Parameters

Graph database config

In [14]:
non_inst_ontology_path = os.path.join("..", "ontology/non_inst", "CRA V16 MFL.ttl")

with open(non_inst_ontology_path, 'r', encoding='utf-8') as file:
        non_inst_ontology = file.read()

In [15]:
inst_ontology_path = os.path.join("..", "ontology/inst", "CRA V16.2 MFL Instantiated Ontology.ttl")

g = Graph()
g.parse(inst_ontology_path)

<Graph identifier=N320a106ba5b64e24a3285b62b92cab7a (<class 'rdflib.graph.Graph'>)>

In [16]:
max_sparql_retries = 5  # Max number of attempts at generating a correct SPARQL query
max_query_retries = 3   # Max number of attempts at generating a SPARQL query that returns at least one result

#### Test graph

In [17]:
print(f"Graph has {len(g)} triples.")

Graph has 9253 triples.


In [ ]:
# Define a general SPARQL query to retrieve any subject, predicate, and object
general_query = """
PREFIX cro: <http://WSP.org/ontology/cro#>
SELECT ?operationName ?criticalRiskScenario
WHERE {
    VALUES ?operationName {"Oyu Tolgoi Copper Mine"} .
    ?operation cro:hasName ?operationName ;
               cro:hasRiskAsessmentReport ?riskAssessmentReport .
    ?riskAssessmentReport cro:hasIdentifiedCriticalRiskScenarios ?criticalRiskScenario .
}
"""

# Run the query
results = g.query(general_query)

# Iterate over the results and print
for row in results:
    print(f"Name: {row.operationName}", f"Scenario: {row.criticalRiskScenario}", sep="\n")

Name: Oyu Tolgoi Copper Mine
Scenario: http://WSP.org/ontology/cro#OTC_critical_risk_assesment_report_2023


In [19]:
len(results)

9253

## Prompts

In [20]:
# Standard prompts

sparql_generation_prompt: BasePromptTemplate = SPARQL_GENERATION_SELECT_PROMPT

sparql_fix_prompt: BasePromptTemplate = GRAPHDB_SPARQL_FIX_PROMPT

qa_prompt: BasePromptTemplate = GRAPHDB_QA_PROMPT

no_result_prompt: BasePromptTemplate = SPARQL_NO_RESULT_PROMPT

combined_prompt: BasePromptTemplate = COMBINED_QA_PROMPT

In [21]:
sparql_generation_chain = sparql_generation_prompt | llm

sparql_fix_chain = sparql_fix_prompt | llm

qa_chain = qa_prompt | llm

no_result_chain = no_result_prompt | llm

combined_chain = combined_prompt | llm

In [22]:
sparql_templates_path = os.path.join("..", "ontology/sparql_templates", "sparql templates.txt")

with open(sparql_templates_path, 'r', encoding='utf-8') as file:
        sparql_templates = file.read()

In [23]:
sparql_templates

'Template #1: Retrieving Sub-Elements of a Specific Operation\n\nTo find the sub-elements of a specific operation named "XXX," you can use the following SPARQL query. Replace "XXX" with the desired operation name:\nPREFIX cro: <http://WSP.org/ontology/cro#>\nSELECT ?riskManagementSubElement \nWHERE {\n    ?operation cro:hasName "XXX" ;\n               cro:hasRiskAsessmentReport ?riskAssessmentReport .\n    ?riskAssessmentReport cro:hasRiskManagementSystem ?riskManagementSystem .\n    ?riskManagementSystem cro:hasRiskManagementSubElement ?riskManagementSubElement .\n}\n\n\n\nTemplate #2: Retrieving Risk Management System of a Specific Operation\n\nTo look at the risk management system associated with a specific operation named "XXX," you can use the following SPARQL query. Replace "XXX" with the operation name you\'re interested in:\nPREFIX cro: <http://WSP.org/ontology/cro#>\nSELECT ?riskManagementSystem\nWHERE {\n   ?operation cro:hasName "XXX" ;\n               cro:hasRiskAsessmentRe

## Classes

In [24]:
_run_manager = CallbackManagerForChainRun.get_noop_manager()
callbacks = _run_manager.get_child()

In [25]:
class SPARQLQueryHandler:
    def __init__(self, max_sparql_retries: int = 3):
        self.max_sparql_retries = max_sparql_retries

    def log_prepared_sparql_query(self, _run_manager, generated_query: str) -> None:
        _run_manager.on_text("Generated SPARQL:", end="\n", verbose=True)
        _run_manager.on_text(generated_query, color="green", end="\n", verbose=True)

    def log_invalid_sparql_query(self, _run_manager, generated_query: str, error_message: str) -> None:
        _run_manager.on_text("Invalid SPARQL query: ", end="\n", verbose=True)
        _run_manager.on_text(generated_query, color="red", end="\n", verbose=True)
        _run_manager.on_text("SPARQL Query Parse Error: ", end="\n", verbose=True)
        _run_manager.on_text(error_message, color="red", end="\n\n", verbose=True)

    def prepare_sparql_query(self, _run_manager, generated_sparql: str) -> str:
        prepareQuery(generated_sparql)
        self.log_prepared_sparql_query(_run_manager, generated_sparql)
        return generated_sparql

    def get_prepared_sparql_query(self, _run_manager, generated_sparql: str, ontology_schema: str) -> str:
        try:
            return self.prepare_sparql_query(_run_manager, generated_sparql)
        except Exception as e:
            retries = 0
            error_message = str(e)
            self.log_invalid_sparql_query(_run_manager, generated_sparql, error_message)
            print("Error message: ", error_message)

            while retries < self.max_sparql_retries:
                try:
                    sparql_fix_chain_result = sparql_fix_chain.invoke(
                        {
                            "error_message": error_message,
                            "generated_sparql": generated_sparql,
                            "schema": ontology_schema,
                        }
                    )
                    generated_sparql = sparql_fix_chain_result.content
                    return self.prepare_sparql_query(_run_manager, generated_sparql)
                except Exception as e:
                    retries += 1
                    parse_exception = str(e)
                    print("Error message (parse_exception): ", parse_exception)
                    self.log_invalid_sparql_query(_run_manager, generated_sparql, parse_exception)

            print("The generated SPARQL query is invalid.")
            return None

    def execute_query(self, g, query: str) -> List[rdflib.query.ResultRow]:
        try:
            rdf_results = g.query(query)

            results_list = []
            for row in rdf_results:
                results_list.append(row)

            return results_list
        except Exception:
            print("Failed to execute the generated SPARQL query.")
            return []

In [ ]:
class VectorStoreHandler:
    def __init__(self, llm, doc_index_name, ontology_index_name, k: int = 5):
        self.llm = llm
        self.k = k
        self.doc_index_name = doc_index_name
        self.ontology_index_name = ontology_index_name
        self.headers = {'Content-Type': 'application/json',
            'api-key': search_api_key}
        self.params = {'api-version': search_api_version}
    
    def perform_vector_search(self, system_prompt: str, input_query: str):
        raw_query = str(system_prompt + input_query)    
        search_query = self.llm.invoke(raw_query)
        
        search_query = str(search_query.content)
        print("### Querying pdf document database: ", search_query)
                    
        # Documents Vector Store        
        search_payload = {
            "search": search_query,
            "queryType": "semantic",
            "vectorQueries": [{"text": search_query, "fields": "embedding", "kind": "text"}],
            "semanticConfiguration": search_sem_config,
            "captions": "extractive",
            "answers": "extractive|count-3",
            "queryLanguage": "en-us",
            "count": "true",
            "top": self.k
        }
        
        resp = requests.post(f"{search_endpoint}/indexes/{self.doc_index_name}/docs/search",
                            data=json.dumps(search_payload), headers=self.headers, params=self.params)
        search_results = resp.json()

        content = dict()
        ordered_content = OrderedDict()

        for result in search_results['value']:
            if result['@search.rerankerScore'] > 1: #Change treshold if needed
                content[result['id']] = {
                    "doc_path": result['doc_path'],
                    "chunk": result['chunk'],
                    "score": result['@search.rerankerScore']
                }

        topk = self.k
        
        count = 0  # To keep track of the number of results added
        for id in sorted(content, key=lambda x: content[x]["score"], reverse=True):
            ordered_content[id] = content[id]
            count += 1
            if count >= topk:  # Stop after adding topK results
                break
            
        index_results = []
        for key, value in ordered_content.items():
            try:
                index_results.append(Document(page_content=value["chunk"], metadata={
                    "doc_path": value['doc_path'], "score": value["score"]}))

            except:
                print("An exception occurred")
    
        return index_results
        
    def perform_graph_search(self, system_prompt: str, input_query: str):
        
        raw_query = str(system_prompt + input_query)    
        search_query = self.llm.invoke(raw_query)
        
        search_query = str(search_query.content)
        print("### Querying ontology database: ", search_query)
        
        # Ontology Vector Store

        search_payload = {
            "search": search_query,
            "queryType": "semantic",
            "vectorQueries": [{"text": search_query, "fields": "embedding", "kind": "text"}],
            "semanticConfiguration": search_sem_config,
            "captions": "extractive",
            "answers": "extractive|count-3",
            "queryLanguage": "en-us",
            "count": "true",
            "filter": "doc_path eq 'CRA V16.2 MFL Instantiated Ontology.ttl'",
            "filter": "doc_path eq 'CRA V17 MFL & NLE Instantiated Ontology.ttl'",
            "top": self.k
        }
        
        resp = requests.post(f"{search_endpoint}/indexes/{self.ontology_index_name}/docs/search",
                            data=json.dumps(search_payload), headers=self.headers, params=self.params)
        search_results = resp.json()

        content = dict()
        ordered_content = OrderedDict()

        for result in search_results['value']:
            if result['@search.rerankerScore'] > 1: #Change treshold if needed
                content[result['id']] = {
                    "doc_path": result['doc_path'],
                    "individual": result['individual'],
                    "score": result['@search.rerankerScore']
                }

        topk = self.k
        
        count = 0  # To keep track of the number of results added
        for id in sorted(content, key=lambda x: content[x]["score"], reverse=True):
            ordered_content[id] = content[id]
            count += 1
            if count >= topk:  # Stop after adding topK results
                break
            
        index_results = []
        for key, value in ordered_content.items():
            try:
                index_results.append(Document(page_content=value["individual"], metadata={
                    "doc_path": value['doc_path'], "score": value["score"]}))

            except:
                print("An exception occurred")
    
        return index_results

## Questions

#### Define questions manually

In [31]:
questions = [
    "List all the Critical Risk Scenarios that impact Cape Lambert", # Q0
    "Provide me with a brief description of Dampier Port", # Q1
    "Describe the 2021 Dredging Campaign carried out at Dampier Port", # Q2
    "Compare the critical risk scenarios of CLO and DPO", # Q3
    "What are the common recommendations between CLO and DPO?", # Q4
    "What equipments are being impacted by the risk with the highest total loss at Dampier Port operation", # Q5
    "What facilities are being impacted by risks at Oyu Tolgoi and  Dampier Port operations", # Q6
    "What is the total business interuption of top 5 risks in Oyu Tolgoi operation", # Q7
    "What are the key information about the critical assessment report  conducted for Dampier Port operation", # Q8
    "what  are the electricity sources of Oyu Tolgoi and  Dampier Port operations", # Q9
    "what  are the common sources of electricity  between Oyu Tolgoi and  Dampier Port operations", # Q10
    "What scenarios exist for cascading risks happening at  Dampier Port operation", # Q11
    "List all equipments with their corresponding critical risk scenrairo at Dampier Port Operation", # Q12
    "List all facilities with their corresponding critical risk scenrairo at Dampier Port Operation", # Q13
    "There is a predicted high temprature at Dampier port operation, list all potentianl critical risk scenarios and recommendations associated along their IDs. Explain how these scenarios are linked to the high temperature", # Q14
    "List all the existing maintenance projects at Dampier port operation", # Q15
    "is there any common recommendations for critical risk scenarios accross the operations you have in your knowledge base", # Q16
    "Give me a comparisson of the critical risk scenarios in Dampier Port Operation and Cape Lambert Operation. What similarities can we observe on both operations?", # Q17
    "Out of DPO and CLO, which operation is more likely to be hit by a cyclone", # Q18
    "What are the top 5 risks in terms of total loss across the sites for Dampier Port, Cape Lambert and Oyu Tolgoi", # Q19
    "What is the recommendation that mitigates the most in terms of total loss across the sites for Dampier Port, Cape Lambert and Oyu Tolgoi. ", # Q20
    "What are the most likely risks across the sites for Dampier Port, Cape Lambert and Oyu Tolgoi. Include the likelihood ranking", # Q21
    "Give me a list of all the risks with likelihood of 'Possible' across the sites for Dampier Port, Cape Lambert and Oyu Tolgoi. ", # Q22
    "What year was the risk Scenario CLO-03 identified?", # Q23
    "What are the main sources of energy for the power station providing power to Dampier Port", # Q24
    "What are the pieces of equipement that are found in large critical risk scenarios. ", # Q25
    "What are the most common types of maintenance being conducted at across the sites in your knowledge base", # Q26
    "What are the main differences between Dampier Port and Cape Lambert", # Q27
    "Is there a single point of failure in Cape Lambert in terms of electricity supply?", # Q28
    "Compare the commonalities of the risks with the highest total loss across the sites for Dampier Port, Cape Lambert and Oyu Tolgoi.", # Q29
    "What are the recommendations with the greatest benefit in terms of return of investment and overall estimated cost", # Q30
    "For Risk Management Systems and Elements: which element or subelement is the strongest for a site, or across sites. Is there a subelement that is impacting the overall score the most? ", # Q31
    "On average, what is the lowest performing Risk Management System across the 3 sites?", # Q32
    "What is the lowest performing Risk Management System on Dampier Port Operations. Include some insights on why this one scored low, and give some ideas on how to improve this Risk Management System.", # Q33
    "How much did the Risk Management System survey percentage changed for the current year compared to the previous year for Dampier Port Operations? ", # Q34
    "What Risk Management Sub Elements showed the greatest increase for the current year compared to the previous survey at Cape Lambert Operations? What insights can you gather from these observations? ", # Q35
    "What Risk Management Sub Elements showed a decrease in their scores the current year compared to the previous survey score? Give some ideas to address these shortcomings." # Q36
]

#### Get questions from xlsx

In [32]:
# Extract questions from an xlsx file and convert to a list

questions_file_path = os.path.join("..", "validation", "LLM Questions - Workshop 241024.xlsx")
questions = pd.read_excel(questions_file_path, header=0)
questions = questions.iloc[:, 1].tolist()

questions

['List all the Critical Risk Scenarios that impact Cape Lambert',
 'Provide me with a brief description of Dampier Port',
 'Describe the 2021 Dredging Campaign carried out at Dampier Port',
 'Compare the critical risk scenarios of CLO and DPO',
 'What are the common recommendations between CLO and DPO?',
 'What equipments are being impacted by the risk with the highest total loss at Dampier Port operation',
 'What facilities are being impacted by risks at Oyu Tolgoi and  Dampier Port operations',
 'What is the total business interuption of top 5 risks in Oyu Tolgoi operation',
 'What are the key information about the critical assessment report  conducted for Dampier Port operation',
 'what  are the electricity sources of Oyu Tolgoi and  Dampier Port operations',
 'what  are the common sources of electricity  between Oyu Tolgoi and  Dampier Port operations',
 'What scenarios exist for cascading risks happening at  Dampier Port operation',
 'List all equipments with their corresponding c

## Retrieve

### Run experiment

In [28]:
# Parameters
sparql_handler = SPARQLQueryHandler(max_sparql_retries=3)
vectorstore_handler = VectorStoreHandler(llm, doc_index_name, ontology_index_name, k=5)

In [29]:
# Create a dataframe to store the results
results_df = pd.DataFrame(columns=["input_query", "generated_sparql", "sparql_results", "PTO_results", "retrieved_docs", "result"])

In [58]:
input_queries = [
"What are the recommendations with the greatest benefit in terms of return of investment and overall estimated cost"
]

In [59]:
for input_query in input_queries:
    
    # SPARQL Query Generation
    vectorstore_handler.k = 5
    ontology_index_results = vectorstore_handler.perform_graph_search(graph_search_prompt, input_query)

    useful_individuals = []
    for result in ontology_index_results:
        useful_individuals.append(result.page_content)
    
    sparql_generation_chain_result = sparql_generation_chain.invoke(
        {"prompt": input_query, "individuals": str(useful_individuals), "schema": non_inst_ontology, "sparql_queries": sparql_templates}
    )

    raw_sparql = sparql_generation_chain_result.content
    generated_sparql = sparql_handler.get_prepared_sparql_query(_run_manager, raw_sparql, non_inst_ontology)
    sparql_results = sparql_handler.execute_query(g, generated_sparql)
        
    # If no results, iterate up to a max number of attempts
    query_retries = 0
    while sparql_results == [] and query_retries < max_query_retries:
        
        query_retries += 1
        
        print(f"### Attempt {query_retries}:", generated_sparql)
        print(f"### No results retrieved by the query, generating a new one...")

        # Use no_result_chain to generate a new query
        no_result_chain_result = no_result_chain.invoke(
            {
                "generated_sparql": generated_sparql,
                "prompt": input_query,
                "schema": non_inst_ontology,
                "individuals": str(useful_individuals),
            }
        )

        # Get the newly generated SPARQL query and execute it
        generated_sparql = no_result_chain_result.content
        generated_sparql = sparql_handler.get_prepared_sparql_query(_run_manager, generated_sparql, non_inst_ontology)
        sparql_results = sparql_handler.execute_query(g, generated_sparql)

    # Handle case when no results were retrieved after max iterations
    if sparql_results == []:
        print(f"### No results from graph database after {max_query_retries} attempts for query: '{input_query}'. Falling back to standard RAG approach.")
    else:
        print("### Successful sparql query:")
        print(generated_sparql)

    vectorstore_handler.k = 5
    # Vector store search
    pdf_index_results = vectorstore_handler.perform_vector_search(pdf_search_prompt, input_query)
    
    # Answer generation
    try:
        combined_chain_result = combined_chain.invoke(
            {
                "question": input_query, 
                "sparql_results": sparql_results,  
                "index_results": pdf_index_results
            }
        )
    except Exception as e:
        print("An error occurred while generating the answer. Falling back to standard RAG.")
        print(e)
        combined_chain_result = combined_chain.invoke(
            {
                "question": input_query, 
                "sparql_results": "### The output of the graph query included too many results and exceeded the context window size.",  
                "index_results": pdf_index_results
            }
        )
        
    result = combined_chain_result.content
    print("### Generated answer:")
    print(result)

    # Store results
    test_result = pd.DataFrame([{
        "input_query": str(input_query),
        "generated_sparql": str(generated_sparql),
        "sparql_results": str(sparql_results),
        "PTO_results": str(ontology_index_results),
        "retrieved_docs": str(pdf_index_results),
        "result": str(result),
    }])

    results_df = pd.concat([results_df, test_result], ignore_index=True)

### Querying ontology database:  Recommendations with the greatest benefit in terms of return on investment and overall estimated cost.
### Successful sparql query:
PREFIX cro: <http://WSP.org/ontology/cro#>
SELECT ?recommendation ?recommendationPriority ?recommendationEstimatedCostUSDM
WHERE {
    ?recommendation cro:recommendationPriority "High Return" ;
                    cro:recommendationEstimatedCostUSDM ?recommendationEstimatedCostUSDM .
}
ORDER BY DESC(?recommendationEstimatedCostUSDM)
### Querying pdf document database:  What recommendations offer the greatest return on investment and the lowest overall estimated cost?
### Generated answer:
The recommendations with the greatest benefit in terms of return on investment and overall estimated cost are as follows:

1. **CLO-22-10-02**: This recommendation has an estimated benefit of 50.0.
2. **DPO-22-08-01**: This recommendation also has an estimated benefit of 50.0.
3. **DPO-22-08-02**: This recommendation has an estimated benef

### Check steps

In [55]:
sparql_results

[(rdflib.term.Literal('Flooding of car dumper 5 vault lower levels has previously been an issue. Reports of two instances in 2013 and 2014 of significant flooding of the CD5C cutting and vault gave rise to concerns of the adequacy and reliability of the stormwater pumping system (inclusive of its electricity supply and backup). This has now been addressed with the advent of a more suitable pumping arrangement. The flooding also occurred in 2019 during a cyclone when the site was evacuated. Plant control logic has been changed to always keep the submersible pump on during a cyclone when the site has been evacuated. And were used effectively in Cyclone Damian and can now be accessed remotely.'),),
 (rdflib.term.Literal('The provision of infrastructure (electricity and water) at CLB is of a reasonable standard. More work is underway. Massive water leaks have been detected. Two separate projects are underway to resolve the water leak issue at CLA and CLB. Multiple failure modes identified.

In [97]:
useful_individuals

[':DPO_Vehicle_Interaction is a NamedIndividual , :RiskManagementSubElement ; :currentSurveyScore 8 ; :previousSurveyScore 8 ; :riskManagementSubElementInformation "Risk Management SubElement Information"; "Critical controls for the Vehicle Interaction Management Plan are audited internally for compliance. Opportunities for improvement are reviewed on a quarterly basis or more frequently. DPO uses CRM for this task. CONFIDENTIALHamersley Iron Pty Ltd - Dampier Port Rio Tinto – Critical Risk Assessment 16 October 2023 Page 73" , "The site has conducted a risk assessment to identify the potential root causes of collisions and rollovers at the site. Control strategies have been developed to mitigate the risk of Vehicle Interaction." , "The site has developed a Management Plan or Procedure to provide a framework for effective management of Vehicle Interaction activities, to minimise any harm to persons, property, the environment, reputation or financial loss to the business." , "The site h

In [109]:
print(textwrap.fill(result, width=150))

The most likely risks across the sites for Dampier Port, Cape Lambert, and Oyu Tolgoi, along with their likelihood rankings, are as follows:  ###
Dampier Port 1. **DPO-04**: Rare 2. **DPO-07**: Rare 3. **DPO-12**: Rare 4. **DPO-23**: Rare 5. **DPO-01**: Unlikely 6. **DPO-05**: Unlikely 7.
**DPO-10**: Unlikely 8. **DPO-11**: Unlikely 9. **DPO-16**: Unlikely 10. **DPO-20**: Unlikely 11. **DPO-22**: Unlikely  ### Cape Lambert Operations 1.
**CLO-02**: Rare 2. **CLO-05**: Rare 3. **CLO-07**: Rare 4. **CLO-10**: Rare 5. **CLO-12**: Rare 6. **CLO-23**: Rare 7. **CLO-24**: Rare 8.
**CLO-03**: Unlikely 9. **CLO-04**: Unlikely 10. **CLO-06**: Unlikely 11. **CLO-09**: Unlikely 12. **CLO-11**: Unlikely 13. **CLO-13**: Unlikely 14.
**CLO-22**: Unlikely  ### Oyu Tolgoi There is no information available regarding the risks and their likelihood rankings for Oyu Tolgoi.  For
additional context, the risks at Dampier Port and Cape Lambert include potential damage from cyclones, flooding, and structural 

### Store results

In [63]:
# Generate question codes
code_sequence = [f"CRD-{str(i).zfill(2)}" for i in range(1, len(results_df) + 1)]
try:
    results_df.insert(0, 'code', code_sequence)
except:
    pass

In [64]:
# Add 'Outcome' and 'Validation' columns
results_df['outcome'] = results_df['sparql_results'].apply(lambda x: 0 if x == "[]" else 1)
positive_outcome_count = results_df['outcome'].sum()
total_outcome_count = len(results_df)
results_df['validation'] = ((positive_outcome_count / total_outcome_count) * 100).round(2)

In [65]:
# Calculate summary statistics
validation_percentage = (positive_outcome_count / total_outcome_count) * 100
failed_questions_df = results_df[results_df['outcome'] == 0]
failed_question_count = len(failed_questions_df)
failed_question_codes = failed_questions_df['code'].tolist()

In [67]:
# Write data to Excel
timestamp = pd.Timestamp.now().strftime("%Y_%m_%d_%H_%M_%S")

output_file_path = os.path.join("..", f"validation/validation_results/Combined RAG Validation_{timestamp}.xlsx")

with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:

    # Save results data to Sheet1
    results_df.to_excel(writer, sheet_name='Sheet1', index=False)

    # Create summary data for Sheet2
    summary_data = {
        'Statistic': ['Accuracy', 'Error Count', 'Error Codes'],
        'Value': [f'{validation_percentage:.2f}%', failed_question_count, ', '.join(str(failed_question_codes))]
    }
    summary_df = pd.DataFrame(summary_data)

    summary_df.to_excel(writer, sheet_name='Sheet2', index=False)

    # Access the workbook and worksheet for formatting
    workbook = writer.book
    worksheet1 = writer.sheets['Sheet1']
    worksheet2 = writer.sheets['Sheet2']

    # Set column width
    worksheet1.set_column(0, len(results_df.columns) - 1, 30)
    worksheet2.set_column(0, 1, 20)

    # Add conditional formatting to the outcome column in Sheet1
    outcome_range = f'G2:F{len(results_df) + 1}'
    format_pass = workbook.add_format({'bg_color': '#77DD77'})
    format_fail = workbook.add_format({'bg_color': '#FF0000'})

    worksheet1.conditional_format(outcome_range, {
        'type': 'cell',
        'criteria': '==',
        'value': 1,
        'format': format_pass
    })

    worksheet1.conditional_format(outcome_range, {
        'type': 'cell',
        'criteria': '==',
        'value': 0,
        'format': format_fail
    })